In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
!git clone https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch.git
!cd Yolov5_DeepSort_Pytorch
!pip install -r requirements.txt


Cloning into 'Yolov5_DeepSort_Pytorch'...
remote: Enumerating objects: 16852, done.
remote: Counting objects: 100% (685/685), done.
remote: Compressing objects: 100% (274/274), done.
remote: Total 16852 (delta 509), reused 411 (delta 411), pack-reused 16167 (from 4)
Receiving objects: 100% (16852/16852), 127.89 MiB | 21.37 MiB/s, done.
Resolving deltas: 100% (10860/10860), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 78.6 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture("Video3.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("output_with_routes3.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

tracker = DeepSort(
    max_age=50,
    n_init=5,
    max_iou_distance=0.5
)

CLASS_NAMES = model.names
TRACK_CLASSES = [0, 1, 2]

trajectories = {}

def get_color(track_id):
    import random
    random.seed(track_id)
    return tuple(random.randint(0, 255) for _ in range(3))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)[0]
    detections = []

    for box, conf, cls in zip(results.boxes.xyxy, results.boxes.conf, results.boxes.cls):
        cls = int(cls)
        if cls in TRACK_CLASSES and conf > 0.4:
            x1, y1, x2, y2 = map(int, box)
            bbox_xywh = [x1, y1, x2 - x1, y2 - y1]
            label = CLASS_NAMES[cls]
            detections.append((bbox_xywh, conf.item(), label))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed() or track.track_id is None:
            continue

        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2

        if track_id not in trajectories:
            trajectories[track_id] = []
        trajectories[track_id].append((cx, cy))

        cv2.rectangle(frame, (x1, y1), (x2, y2), get_color(track_id), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    for track_id, points in trajectories.items():
        color = get_color(track_id)
        for i in range(1, len(points)):
            cv2.line(frame, points[i - 1], points[i], color, 2)

    out.write(frame)

cap.release()
out.release()

In [ ]:
from google.colab import files
files.download('output_with_routes3.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>